In [9]:
from emc.workflows.correct import init_emc_wf
from pathlib import Path
import nipype.pipeline.engine as pe
from nipype.interfaces import utility as niu
from dipy.data import get_fnames
from dmriprep.workflows.dwi.util import init_dwi_reference_wf
from dmriprep.interfaces.vectors import CheckGradientTable

#fdata, fbvals, fbvecs = get_fnames("sherbrooke_3shell")
#fdata, fbvals, fbvecs = get_fnames("stanford_hardi")
fdata = '/Users/derekpisner/Downloads/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.nii.gz'
fbvals = '/Users/derekpisner/Downloads/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bval'
fbvecs = '/Users/derekpisner/Downloads/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bvec'

In [10]:
# Create a workflow that can be added as a node to
# `~dmriprep.workflows.dwi.base.init_dwi_preproc_wf` and (for now) envelops
# `~dmriprep.workflows.dwi.util.init_dwi_reference_wf`
def build_emc_workflow(fdata, fbvals, fbvecs, work_dir='/tmp'):

    wf_emc = pe.Workflow(name='emc_correction')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_files', 'in_bval',
                                                      'in_bvec']),
        name='inputnode')

    inputnode.inputs.dwi_file = fdata
    inputnode.inputs.in_bval = fbvals
    inputnode.inputs.in_bvec = fbvecs

    emc_wf_node = init_emc_wf('emc_wf')

    wf_emc.connect([
        (inputnode, emc_wf_node, [('dwi_file', 'meta_inputnode.dwi_file'),
                                  ('in_bval', 'meta_inputnode.in_bval'),
                                  ('in_bvec', 'meta_inputnode.in_bvec')])
    ])

    wf_emc.base_dir = work_dir

    cfg = dict(execution={'stop_on_first_crash': False,
                          'parameterize_dirs': True, 'crashfile_format': 'txt',
                          'remove_unnecessary_outputs': False,
                          'poll_sleep_duration': 0, 'plugin': 'MultiProc'})
    for key in cfg.keys():
        for setting, value in cfg[key].items():
            wf_emc.config[key][setting] = value

    return wf_emc

In [ ]:
wf = build_emc_workflow(fdata, fbvals, fbvecs)
res = wf.run(plugin='MultiProc')

210305-18:22:05,607 nipype.workflow INFO:
	 Workflow emc_correction settings: ['check', 'execution', 'logging', 'monitoring']
210305-18:22:05,661 nipype.workflow INFO:
	 Running in parallel.
210305-18:22:05,666 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210305-18:22:06,39 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 28.40/28.80, Free processors: 10/12.
                     Currently running:
                       * emc_correction.emc_wf.emc_vectors_node
                       * emc_correction.emc_wf.patch2self_node
210305-18:22:15,323 nipype.workflow INFO:
	 [Job 3] Completed (emc_correction.emc_wf.emc_vectors_node).
210305-18:22:15,327 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 11/12.
                     Currently running:
                       * emc_correction.emc_wf.patch2self_nod

In [ ]:
list(res.nodes())

[emc_correction.emc_wf.emc_vectors_node,
 emc_correction.emc_wf.extract_b0_node,
 emc_correction.emc_wf.split_b0s_node,
 emc_correction.emc_wf.split_dwis_node,
 emc_correction.emc_wf.match_transforms_node,
 emc_correction.emc_wf.b0_based_vector_transforms,
 emc_correction.emc_wf.b0_based_image_transforms,
 emc_correction.emc_wf.merge_b0s_node,
 emc_correction.emc_wf.b0_median,
 emc_correction.emc_wf.prune_b0s_from_dwis_node,
 emc_correction.emc_wf.uncorrect_model_images,
 emc_correction.emc_wf.merge_EMC_corrected_dwis_node,
 emc_correction.emc_wf.dwi_reference_wf.validate,
 emc_correction.emc_wf.dwi_reference_wf.extract_b0,
 emc_correction.emc_wf.dwi_reference_wf.reg_b0,
 emc_correction.emc_wf.dwi_reference_wf.pre_mask,
 emc_correction.emc_wf.dwi_reference_wf.rescale_b0,
 emc_correction.emc_wf.dwi_reference_wf.enhance_and_skullstrip_dwi_wf.n4_correct,
 emc_correction.emc_wf.dwi_reference_wf.enhance_and_skullstrip_dwi_wf.fixhdr_unifize,
 emc_correction.emc_wf.dwi_reference_wf.enhance_an